In [7]:
import time
from selenium import webdriver

driver = webdriver.Chrome('C:/Users/user/jupyternb/chromedriver_win32/chromedriver.exe')  # Optional argument, if not specified will search path.
driver.get('https://www.google.co.jp/search?num=100');
time.sleep(3) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('python スクレイピング')
search_box.submit()
time.sleep(3) # Let the user actually see something!

url = driver.current_url
print(url)



driver.quit()

https://www.google.co.jp/search?num=100&source=hp&ei=O5m4XoSCAcHj-Ab9mL_YDA&q=python+%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0


In [11]:
from selenium import webdriver
import csv
import time
import requests
from bs4 import BeautifulSoup
import lxml

# Chromeを立ち上げる
# ローカル環境での「chromedriver.exe」のパスを指定する
chrome = webdriver.Chrome("C:/Users/user/jupyternb/chromedriver_win32/chromedriver.exe")
chrome.get('https://www.google.co.jp/')

# 検索キーワードの指定・・・ "python スクレイピング" のように２語以上の場合は、半角スペースを追加する
keyword = "python スクレイピング 自動化"
# 取得ページの指定数
per = "10" 

# 検索実行
search_box = chrome.find_element_by_name('q')
search_words = keyword
search_box.send_keys(search_words)
search_box.submit()

# 3秒間スリープ
time.sleep(3)

# 検索を実行したURLを取得する
cur_url = chrome.current_url
print(cur_url)

# データの格納の準備する
result = {}
title_text_value = []
disc_text_value = []
url_text_value = []

# 関数
def response_get():
    # ヘッダー情報を指定する
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    }

    r = requests.get(cur_url, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    
    # 検索結果のタイトルとリンクを取得
    link_title = soup.select('.r > a')

    # 検索結果の説明部分を取得
    link_disc = soup.select('.s > div > .st')
    
    if(len(link_disc) <= len(link_title)):
        leng = len(link_disc)
        for i in range(leng):
            try:
                # タイトルのテキスト部分のみを取得
                title_text = link_title[i].get_text()
                title_text_value.append(title_text)
                result["TITLE"] = title_text_value

                # 説明のテキスト部分のみを取得／余分な改行コードを削除する
                t01 = link_disc[i].get_text()
                t02 = t01.replace('\n', '')
                disc_text = t02.replace('\r', '')
                disc_text_value.append(disc_text)
                result["DESCRIPTION"] = disc_text_value 

                # リンクのみを取得し、余分な部分を削除する
                url_text = link_title[i].get('href').replace('/url?q=','')
                url_text_value.append(url_text)
                result["URL_LINK"] = url_text_value                           
               
                # 次ページ取得・遷移
                next = chrome.find_element_by_css_selector("#foot table td.cur + td a")
                next.click()
            except:
                continue

            # 5秒間スリープ
            time.sleep(5)  

    else:
        leng = len(link_title)
        print("取得おわり")


# 検索結果を取得する
for i in range(int(per)):
    response_get()

# chromeを閉じる
chrome.close()

print(result)


# 検索結果をCSVに出力する
r = zip(*result.values())
fieldnames = ['TITLE', 'DESCRIPTION', 'URL_LINK']

with open('./output/[' + keyword + ']search_selenium_google_it.csv', 'w', newline='', encoding='utf-8') as f:
    dict_writer = csv.writer(f)
    dict_writer.writerow(fieldnames)
    for d in r:
        dict_writer.writerow(d)


https://www.google.co.jp/search?source=hp&ei=TSC5XsTVJKOh-Qan-ZC4DQ&q=python+%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0+%E8%87%AA%E5%8B%95%E5%8C%96
{'TITLE': ['pythonで自動入力、スクレイピングをしてみる - Qiitaqiita.com › Python', '【完全版】PythonとSeleniumでブラウザを自動操作(クローリング ...tanuhack.com › Python › Selenium', '【Python入門】スクレイピングを使って自動的にWebページから ...blog.codecamp.jp › 子ども向け', 'Pythonでブラウザを自動操作する方法 | ガンマソフト株式会社gammasoft.jp › python-browser-automation-by-selenium', 'PythonとSeleniumを使ったブラウザ自動操作 – 名古屋のWeb ...www.inet-solutions.jp › 技術関連情報', '【保存版】Pythonでスクレイピングする方法を初心者向けに徹底 ...dividable.net › python › python-scraping', 'Pythonで毎日圧倒的に効率化している業務自動化術5選 【非 ...dividable.net › python › python-automation', 'スクレイピングツール30選｜初心者でもWebデータを抽出できる ...www.octoparse.jp › top-30-free-web-scraping-software', '業界初！Webデータを自動収集できるWebスクレイピング ...www.octoparse.jp › blog › a-revolutionary-web-scrapin...', 'Pythonの自動化でどんなことができるのかその全てを紹介する | 侍 ...www.sejuku.net › blog', 'pythonで自動入力、スクレイピングをしてみる - Qiitaqiita.com › Python'